In [0]:
catalog_name = "dmc_tallerfinaldatabricks_mariovento"

schema_silver = "silver"
schema_gold = "gold"

In [0]:
from pyspark.sql.functions import current_timestamp, col

In [0]:
# Carga la tabla 'clientes' del esquema 'silver' en un DataFrame de Spark
clientes_silver = spark.table(f"{catalog_name}.{schema_silver}.clientes")

# Crea el DataFrame 'dim_cliente' agregando columnas y seleccionando campos relevantes
dim_cliente = (
    clientes_silver
    .withColumn("sk_cliente", col("id_cliente"))  # Crea la clave sustituta 'sk_cliente' usando 'id_cliente'
    .withColumn("creation_date", current_timestamp())  # Agrega la columna 'creation_date' con la fecha/hora actual
    .select(
        col("sk_cliente"), # Clave sustituta del cliente
        col("id_cliente"), # ID original del cliente
        col("nombre"), # Nombre del cliente
        col("email"), # Email del cliente
        col("ciudad"), # Ciudad del cliente
        col("fecha_registro"), # Fecha de registro del cliente
        col("creation_date") # Fecha/hora de creación del registro en la dimensión
    )
    .orderBy(col("id_cliente").asc())
)

In [0]:
dim_cliente.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_gold}.dim_cliente")

In [0]:
%sql
select * from dmc_tallerfinaldatabricks_mariovento.gold.dim_cliente